# Setup

In [3]:
#@title ## `pip install`
# Don't forget to restart runtime after installing
%pip install "labelbox[data]" --quiet  # installs all required libraries plus extras required in manipulating annotations (shapely, geojson, numpy, PILLOW, opencv-python, etc.)
%pip install -U kaleido  --quiet # for saving the still figures besides .eps (i.e png, pdf)
%pip install poppler-utils  --quiet   # for exporting to .eps extension
%pip install plotly==5.13.0    # need ≥5.6 to use ticklabelstep argument, ≥5.8 to use minor ticks. Release history here https://github.com/plotly/plotly.py/releases

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 37.7 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [2]:
#@title ## Base imports
import os
import cmd
import sys
import json
import numpy as np
import pandas as pd
import scipy
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

import skimage
import skimage.io
import PIL
import PIL.Image
import requests

import labelbox

import IPython.display
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots

# Display versions of python packages
pip_versions = %system pip freeze  # uses colab magic to get list from shell
pip_versions_organized = {
    "standard": [pip_version for pip_version in pip_versions if "==" in pip_version],
    "other": [pip_version for pip_version in pip_versions if "==" not in pip_version]
    }
print(f"Python version: {sys.version} \n")  # display version of python itself (i.e. 3.8.10)
cli = cmd.Cmd()
cli.columnize(pip_versions_organized["standard"], displaywidth=800)
cli.columnize(pip_versions_organized["other"], displaywidth=160)

Python version: 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0] 

absl-py==1.4.0                backcall==0.2.0         charset-normalizer==2.0.12  cvxopt==1.3.0           dm-tree==0.1.8            fastdownload==0.0.7        GDAL==3.3.2                       google-cloud-bigquery==3.4.2           greenlet==2.0.2           htmlmin==0.1.12              imutils==0.5.4           jupyter-client==6.1.12        libclang==15.0.6.1          mistune==0.8.4           nbclient==0.7.2                  opencv-python==4.6.0.66           parso==0.8.3         pluggy==0.7.1              psutil==5.9.4              pydot-ng==2.0.0     PySocks==1.7.1                      qdldl==0.1.5.post3              Send2Trash==1.8.0        sphinxcontrib-applehelp==1.0.4        tblib==1.7.0                          terminado==0.17.1     traitlets==5.7.1          Werkzeug==2.2.3          
alabaster==0.7.13             backoff==1.10.0         click==8.1.3                cvxpy==1.3.0            docutils==0.16            

In [60]:
#@title Basic helper functions
colab_ip = %system hostname -I   # uses colab magic to get list from shell
colab_ip = colab_ip[0].strip()   # returns "172.28.0.12"
# Get most possible port names with: !sudo lsof -i -P -n | grep LISTEN
colab_port = 9000                # could use 6000, 8080, or 9000

notebook_filename = filename = requests.get(f"http://{colab_ip}:{colab_port}/api/sessions").json()[0]["name"]

# Avoids scroll-in-the-scroll in the entire Notebook
def resize_colab_cell():
  display(IPython.display.Javascript("google.colab.output.setIframeHeight(0, true, {maxHeight: 10000})"))
get_ipython().events.register("pre_run_cell", resize_colab_cell)


#@markdown ### func `def get_path_to_save(...):`
def get_path_to_save(plot_props:dict=None, file_prefix="", save_filename:str=None, save_in_subfolder:str=None, extension="jpg", dot=".", create_folder_if_necessary=True):
    """Code created myself (Rahul Yerrabelli)"""
    replace_characters = {
        "$": "",
        "\\frac":"",
        "\\mathrm":"",
        "\\left(":"(",  "\\right)":")",
        "\\left[":"[",  "\\right]":"]",
        "\\": "",       "/":"-",
        "{": "(",       "}": ")",
        "<":"",         ">":"",
        "?":"",
        "_":"",
        "^":"",
        "*":"",
        "!":"",
        ":":"-",
        "|":"-",
        ".":"_",
    }

    # define save_filename based on plot_props
    if save_filename is None:
        save_filename = "unnamed"

    save_path = [
                 "outputs",
                f"""{notebook_filename.split(".",1)[0]}""",
                ]
    if save_in_subfolder is not None:
        if isinstance(save_in_subfolder, (list, tuple, set, np.ndarray) ):
            save_path.append(**save_in_subfolder)
        else:  # should be a string then
            save_path.append(save_in_subfolder)
    save_path = os.path.join(*save_path)

    if not os.path.exists(save_path) and create_folder_if_necessary:
        os.makedirs(save_path)
    return os.path.join(save_path, file_prefix+save_filename+dot+extension)


<IPython.core.display.Javascript object>

In [61]:
#@title ## Mount google drive and import my code

mountpoint_folder_name = "drive"  # can be anything, doesn't have to be "drive"
project_path_within_drive = "PythonProjects/SpeculumAnalysis" #@param {type:"string"}
project_path_full = os.path.join("/content/",mountpoint_folder_name,
                        "MyDrive",project_path_within_drive)

%cd {project_path_full}

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/drive/MyDrive/Computer Backups/Rahul Yerrabelli drive/PythonProjects/SpeculumAnalysis


In [62]:
try:
    import google.colab.drive
    import os, sys
    # Need to move out of google drive directory if going to remount
    %cd
    # drive.mount documentation can be accessed via: drive.mount?
    #Signature: drive.mount(mountpoint, force_remount=False, timeout_ms=120000, use_metadata_server=False)
    google.colab.drive.mount(os.path.join("/content/",mountpoint_folder_name), force_remount=True)  # mounts to a folder called mountpoint_folder_name

    if project_path_full not in sys.path:
        pass
        #sys.path.insert(0,project_path_full)
    %cd {project_path_full}
    
except ModuleNotFoundError:  # in case not run in Google colab
    import traceback
    traceback.print_exc()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/root
Mounted at /content/drive
/content/drive/MyDrive/Computer Backups/Rahul Yerrabelli drive/PythonProjects/SpeculumAnalysis


# Skip ahead from loaded code

In [88]:
speculum_df_raw = pd.read_pickle("data/v2/02_intermediate/speculum_df_raw"+".pkl")
speculum_df_notfailed = pd.read_pickle("data/v2/02_intermediate/speculum_df_notfailed"+".pkl")

labels_df = pd.read_csv("data/v2/02_intermediate/labels_df.csv", index_col=0)
#with open("data/02_intermediate/label_from_id_dict"+".json", "r") as infile:
#    label_from_id_dict = json.load(infile)
    
df_long = pd.read_pickle(  "data/v2/03_processed/combined_df_long.pkl")
df_wide = pd.read_pickle(  "data/v2/03_processed/combined_df_wide.pkl")
df_wide_flat = pd.read_pickle("data/v2/03_processed/combined_df_wide_flat.pkl")

df_agg_long = pd.read_pickle("data/v2/04_aggregated/combined_df_agg_long.pkl")
df_agg_long_flat = pd.read_pickle("data/v2/04_aggregated/combined_df_agg_long_flat.pkl")

df_multiindex = pd.read_pickle("data/v2/03_processed/combined_df_multiindex"+".pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Set up for displaying

## Setup dicts and helper functions

In [104]:
from pandas.core import series
category_orders={
    #"Size": ["S", "M", "L","Unspecified","None"],
    #"Size": ["S","Small", "M", "Medium", "L", "Large","Unspecified","None"],
    "Size": ["Small", "Medium", "Large","Unspecified","None", "S", "M", "L"],  # change the order of S vs Small etc changes the color
    "Material":["Nitrile","Vinyl","Trojan", "Lifestyle", "Durex", "Skyn","None"],
    "Material Type":["Glove","Condom","None"],
    "Method":["Middle","Two","Palm","Middle finger","Two fingers","Palm","Precut","None"],
    "Speculum Type":["Yellow","White","Green","Blue"]
    }
labels = {
    "Trial":            "<b>Trial #</b>",
    "wd_rel":           "<b>Relative Obstruction</b>",
    "wd_rel.mean":      "<b>Mean Relative Obstruction (S.E.)</b>", 
    "mmHg":             "<b>Pressure (mmHg)</b>", 
    "Material":         "<b>Material</b>", 
    "Material Type":    "<b>Material Type</b>",
    "Size":             "<b>Size</b>",
    "Method":           "<b>Method</b>",
    "Brand":            "<b>Brand</b>",
    "Day Ct":           "<b>Day Ct</b>",
    "Set Ct":           "<b>Set Ct</b>",
    "Day Set Ct":       "<b>Day Set Ct</b>",
}

color_discrete_map = {
    "Medium":           px.colors.qualitative.Safe[1],
    "Nitrile":          px.colors.qualitative.Safe[1],
    "Middle finger":    px.colors.qualitative.Safe[1],
    "Small":            px.colors.qualitative.Safe[0],
    "Large":            px.colors.qualitative.Safe[2],
    "Two fingers":      px.colors.qualitative.Safe[4],
    "Vinyl":            px.colors.qualitative.Safe[6],

    "Trojan":           px.colors.qualitative.Safe[7], 
    "Lifestyle":        px.colors.qualitative.Safe[3],
    "Durex":            px.colors.qualitative.Safe[9],
    "Skyn":             px.colors.qualitative.Safe[5],
    "None":             px.colors.qualitative.Safe[6],

    "Yellow": "yellow",
    "White": "grey",
    "Green": "green",
    "Blue": "blue",
}
# pattern shape options =  [ "", "/", "\\", "x", "-", "|", "+", "." ]
pattern_shape_map = {
    "Medium":           "+",
    "Nitrile":          "+",
    "Middle finger":    "+",
    "Small":            "/",
    "Large":            "\\",
    "Vinyl":            "x",
    "Two fingers":      "|",

    "Trojan":           "/", 
    "Lifestyle":        "-", 
    "Durex":            ".", 
    "Skyn":             "\\",
    "None":             "",
}


def criteria_to_str(criteria:dict) -> str:
    return ", ".join([f"{labels.get(key) or key}={val}".replace("<br>","").replace("<b>","").replace("</b>","") for key,val in criteria.items()])


def filter_by_criteria(criteria:dict, starting_df:pd.DataFrame) -> pd.DataFrame:
    #df_sampled = df_agg_long_flat.loc[ np.all([df_agg_long[arg]==val for arg, val in criteria.items()], axis=0) ]
    #df_sampled = df_agg_long_flat.loc[ np.all([ (type(val)!=list and df_agg_long[arg]==val ) or np.in1d(df_agg_long[arg],val)  for arg, val in criteria.items()], axis=0) ]
    #starting_df.loc[ np.all([ (type(val)!=list and starting_df[arg]==val ) or np.in1d(starting_df[arg],val)  for arg, val in criteria.items()], axis=0) ]
    conditions = []
    for arg, val in criteria.items():
        if hasattr(val,"__iter__") and not isinstance(val,str):
            conditions.append( np.in1d(starting_df[arg],val) )
        else:
            print(starting_df[arg], val)
            
            series = starting_df[arg]  # starting_df[arg] is a pd.Series
            conditions.append( ser==val | ser.astype(str)==str(val) )  #  
    
    return starting_df.loc[ np.all(conditions, axis=0) ]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Setup plotly figure saving

In [65]:
default_plotly_save_scale = 4
def save_plotly_figure(fig, file_name:str, animated=False, scale=None, save_in_subfolder:str=None, extensions=None):
    """
    - for saving plotly.express figures only - not for matplotlib
    - fig is of type plotly.graph_objs._figure.Figure,
    - Requires kaleido installation for the static (non-animated) images
    """    
    if scale is None:
        scale = default_plotly_save_scale
    if extensions is None:
        extensions = ["html"]
        if not animated:
            # options = ['png', 'jpg', 'jpeg', 'webp', 'svg', 'pdf', 'eps', 'json']
            extensions += ["eps","png","pdf"]

    for extension in extensions:
        try:
            if extension in ["htm","html"]:
                #fig.update_layout(title=dict(visible=False))
                fig.write_html( get_path_to_save(save_filename=file_name, save_in_subfolder=save_in_subfolder, extension=extension), 
                    full_html=False,
                    include_plotlyjs="directory" )
            else:
                #if extension == "png":
                #    fig.update_layout(title=dict(visible=False))
                fig.write_image(get_path_to_save(save_filename=file_name, save_in_subfolder=save_in_subfolder, extension=extension), scale=scale)
        except ValueError as exc:
            import traceback
            #traceback.print_exception()

#col_options = {col_name:pd.unique(df_long[col_name]).tolist() for col_name in consistent_cols}
#display(col_options)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Setup for plotting aggregates

In [66]:
def customize_figure(fig, width=640, height=360, by_mmHg=True, br_ct=1, space_ct=1, textposition="inside", textfont_color=None) -> dict:
    """ - for plotly figures only. """
    
    if by_mmHg:
        fig.update_xaxes( #tickprefix="At ",   # Dr. WJ and Ashkhan didn't like it
                         ticksuffix="mmHg", showtickprefix="all", showticksuffix="all", tickfont=dict(size=16),
                        mirror=True, linewidth=2, 
                        title=dict(text="<b>Applied Circumferential Pressure</b>", font=dict(size=20, family="Arial Black")),
                        )
        fig.update_yaxes(tickformat=".0%", tickwidth=2,  nticks=21, ticklabelstep=4,
                        mirror="ticks", linewidth=2, range=(0,1), 
                        title=dict(text="<b>Obstruction of<br>Field of View (S.E.)</b>",font=dict(size=18, family="Arial Black")), 
                        #title=dict(text="Width Obstructed of<br>Field of View (S.E.)",font=dict(size=18, family="Arial Black")), 
                        showgrid=True, gridcolor="#DDD", 
                        showspikes=True, spikemode="across", spikethickness=2, spikedash="solid", # ticklabelposition="inside top",
                        )
    #fig.update_traces(textangle=0, textposition="outside", cliponaxis=False)
    fig.update_layout(
        font=dict(
            family="Arial",
            size=16,
            color="black",
        ),
        title={
            "y":1,
            "x":0.5,
            "xanchor": "center",
            "yanchor": "top",
            "font":dict(size=16)
        }, 
        width=width, height=height,
        margin=dict(l=20, r=20, t=20, b=20),
        legend=dict(
            title={"font_family": "Arial Black",},
            yanchor="middle",
            y=0.5,
            xanchor="center",
            x=0.08,
            #bgcolor="LightSteelBlue",
            bordercolor="Black", #font_size=16,
            borderwidth=2,
        ), 
        bargap=0.05, bargroupgap=0.0,
        dragmode="drawopenpath",
        newshape_line_color="cyan",
    )

    if textfont_color is None:
        if isinstance(textposition, (list, tuple, set, np.ndarray, pd.Series) ):
            textfont_color = ["#FFF" if textposition_each == "inside" else "#000" for textposition_each in textposition]
            print(textfont_color)
        elif textposition == "inside":
            textfont_color="#FFF"
        else:
            textfont_color="#000"
    fig.update_traces(textfont_size=16, textangle=0, textfont_color=textfont_color, 
                      textposition=textposition, cliponaxis=False, #textfont_family="Courier",
                      marker_line_color="#000", marker_line_width=2
                    )
    if by_mmHg:
        if textposition == "inside":
            fig.update_traces(texttemplate=[None]+[("&nbsp;"*space_ct)+("<br>"*br_ct)+"<b>%{y:.1%}</b>"]*5,)
        else:
            fig.update_traces(texttemplate=[None]+["<b>%{y:.1%}</b>"+("<br>"*br_ct)+("&nbsp;"*space_ct)]*5,)
            

    config = {
        "toImageButtonOptions" : {
            "format": "png", # one of png, svg, jpeg, webp
            "filename": 'custom_image',
            "scale": default_plotly_save_scale # Multiply title/legend/axis/canvas sizes by this factor
        },
        "modeBarButtonsToAdd": ["drawline","drawopenpath","drawclosedpath","drawcircle","drawrect","eraseshape"]
    }

    return config






<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Plotting

## Plot varying Speculum Type

In [90]:
for col in [ "Spec Ang", "Spec Ht"]:
    df_agg_long_flat[col] = df_agg_long_flat[col].astype(str)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
"""df_agg_long_flat.columns = [ col.replace("wd.","width.") for col in df_agg_long_flat.columns]
df_agg_long_flat.columns = [ col.replace("ht.","wd.") for col in df_agg_long_flat.columns]
df_agg_long_flat.columns = [ col.replace("width.","ht.") for col in df_agg_long_flat.columns]

df_agg_long_flat.columns = [ col.replace("wd_rel.","width_rel.") for col in df_agg_long_flat.columns]
df_agg_long_flat.columns = [ col.replace("ht_rel.","wd_rel.") for col in df_agg_long_flat.columns]
df_agg_long_flat.columns = [ col.replace("width_rel.","ht_rel.") for col in df_agg_long_flat.columns]"""

<IPython.core.display.Javascript object>

In [105]:
criteria = {"Day Ct": 1, "Material":"Nitrile", "Method":"Middle"}
varying = "Size"

df_sampled = filter_by_criteria(criteria,df_agg_long_flat)


fig = px.bar(df_sampled, 
             x="mmHg",y="wd_rel.mean", error_y="wd_rel.sem",
             color=varying, pattern_shape=varying, 
             color_discrete_map=color_discrete_map, pattern_shape_map=pattern_shape_map,
             barmode="group", #text=[".1%<br><br> " for a in range(18)],
             hover_data=["Speculum Type","Day Ct"],
             title=f"Varying {varying} with " + criteria_to_str(criteria), 
             category_orders=category_orders, labels=labels, template="simple_white", 
             )
config = customize_figure(fig, width=1100, height=300)

fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )

fig.show(config=config)
#fig.update_layout(title=dict(text=""))
#save_plotly_figure(fig, file_name=f"Fig 4- Across {varying}- " + criteria_to_str(criteria) )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0      1
1      1
2      1
3      1
4      1
      ..
163    4
164    4
165    4
166    4
167    4
Name: Day Ct, Length: 168, dtype: int64 1
0      Nitrile
1      Nitrile
2      Nitrile
3      Nitrile
4      Nitrile
        ...   
163    Nitrile
164    Nitrile
165    Nitrile
166    Nitrile
167    Nitrile
Name: Material, Length: 168, dtype: object Nitrile
0      Middle
1      Middle
2      Middle
3      Middle
4      Middle
        ...  
163    Middle
164    Middle
165    Middle
166    Middle
167    Middle
Name: Method, Length: 168, dtype: object Middle


In [92]:
#criteria = {"Size":["M","None"], "Method":["Middle","None"]}
criteria = {"Size":"M", "Method":"Middle", "Speculum Type":"Green"}

varying = "Day Ct"

df_sampled = filter_by_criteria(criteria,df_agg_long_flat)
df_sampled["Day Ct"] = df_sampled["Day Ct"].astype(str)
fig = px.bar(df_sampled, 
             x="mmHg",y="wd_rel.mean", error_y="wd_rel.sem", 
             color=varying, pattern_shape=varying, 
             #color_discrete_sequence=px.colors.qualitative.Safe[6:-1:], pattern_shape_sequence=["x", "+", "\\"], 
             color_discrete_map=color_discrete_map, pattern_shape_map=pattern_shape_map,
             barmode="group", #text=[".1%<br><br> " for a in range(18)],
             hover_data=["Size","Material","Method"],
             title=f"Varying {varying} with " + criteria_to_str(criteria), 
             category_orders=category_orders, labels={**labels,"Material":"<b>Glove<br>Material</b>"}, template="simple_white", 
             )

config = customize_figure(fig, width=1100, height=300, textposition="outside", space_ct=3)

#fig.update_traces(texttemplate=["<br>"+"<b>%{y:.1%}</b>", "<b>%{y:.1%}</b>"+("&nbsp;"*10)+("<br>"*1)]*5)

fig.for_each_trace( lambda trace: trace.update(
    textfont_color="#000",
    textposition="outside",
    cliponaxis=False,
    texttemplate=[None]+["&nbsp;"*10+"<b>%{y:.1%}</b><br> "]*5,
    ) if trace.name == "Vinyl" else (), )

fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )


fig.show(config=config)
#fig.update_layout(title=dict(text=""))
#save_plotly_figure(fig, file_name=f"Fig 5- Across {varying}- " + criteria_to_str(criteria) )df_sampled

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-92-8d7c35cf8665>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [94]:
#criteria = {"Size":["M","None"], "Method":["Middle","None"]}
criteria = {"Size":"M", "Material": "Nitrile", "Method":"Middle", "Speculum Type":"White"}

varying = "Day Ct"

df_sampled = filter_by_criteria(criteria,df_agg_long_flat)
fig = px.bar(df_sampled, 
             x="mmHg",y="wd_rel.mean", error_y="wd_rel.sem", 
             color=varying, pattern_shape=varying, 
             #color_discrete_sequence=px.colors.qualitative.Safe[6:-1:], pattern_shape_sequence=["x", "+", "\\"], 
             color_discrete_map=color_discrete_map, pattern_shape_map=pattern_shape_map,
             barmode="group", #text=[".1%<br><br> " for a in range(18)],
             hover_data=["Size","Material","Method"],
             title=f"Varying {varying} with " + criteria_to_str(criteria), 
             category_orders=category_orders, labels={**labels,"Material":"<b>Glove<br>Material</b>"}, template="simple_white", 
             )

config = customize_figure(fig, width=1100, height=300, textposition="outside", space_ct=3)

#fig.update_traces(texttemplate=["<br>"+"<b>%{y:.1%}</b>", "<b>%{y:.1%}</b>"+("&nbsp;"*10)+("<br>"*1)]*5)

fig.for_each_trace( lambda trace: trace.update(
    textfont_color="#000",
    textposition="outside",
    cliponaxis=False,
    texttemplate=[None]+["&nbsp;"*10+"<b>%{y:.1%}</b><br> "]*5,
    ) if trace.name == "Vinyl" else (), )

fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )


fig.show(config=config)
#fig.update_layout(title=dict(text=""))
#save_plotly_figure(fig, file_name=f"Fig 5- Across {varying}- " + criteria_to_str(criteria) )df_sampled

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [106]:
#criteria = {"Material":["Nitrile","None"], "Method":["Middle","None"]}
criteria = {"Day Ct": 4, "Size":"7"}
varying = "Brand"

df_sampled = filter_by_criteria(criteria,df_agg_long_flat)

fig = px.bar(df_sampled, 
             x="mmHg",y="wd_rel.mean", error_y="wd_rel.sem",
             color=varying, pattern_shape=varying, 
             color_discrete_map=color_discrete_map, pattern_shape_map=pattern_shape_map,
             barmode="group", #text=[".1%<br><br> " for a in range(18)],
             hover_data=["Speculum Type","Day Ct"],
             title=f"Varying {varying} with " + criteria_to_str(criteria), 
             category_orders=category_orders, labels=labels, template="simple_white", 
             )
config = customize_figure(fig, width=1600, height=600)

fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )

fig.show(config=config)
#fig.update_layout(title=dict(text=""))
#save_plotly_figure(fig, file_name=f"Fig 4- Across {varying}- " + criteria_to_str(criteria) )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0      1
1      1
2      1
3      1
4      1
      ..
163    4
164    4
165    4
166    4
167    4
Name: Day Ct, Length: 168, dtype: int64 4
0          M
1          M
2          M
3          M
4          M
       ...  
163    8-8.5
164    8-8.5
165    8-8.5
166    8-8.5
167    8-8.5
Name: Size, Length: 168, dtype: object 7


In [107]:
#criteria = {"Material":["Nitrile","None"], "Method":["Middle","None"]}
criteria = {"Day Ct": 4, "Size":"8"}
varying = "Brand"

df_sampled = filter_by_criteria(criteria,df_agg_long_flat)

fig = px.bar(df_sampled, 
             x="mmHg",y="wd_rel.mean", error_y="wd_rel.sem",
             color=varying, pattern_shape=varying, 
             color_discrete_map=color_discrete_map, pattern_shape_map=pattern_shape_map,
             barmode="group", #text=[".1%<br><br> " for a in range(18)],
             hover_data=["Speculum Type","Day Ct"],
             title=f"Varying {varying} with " + criteria_to_str(criteria), 
             category_orders=category_orders, labels=labels, template="simple_white", 
             )
config = customize_figure(fig, width=1600, height=500)

fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )

fig.show(config=config)
#fig.update_layout(title=dict(text=""))
#save_plotly_figure(fig, file_name=f"Fig 4- Across {varying}- " + criteria_to_str(criteria) )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0      1
1      1
2      1
3      1
4      1
      ..
163    4
164    4
165    4
166    4
167    4
Name: Day Ct, Length: 168, dtype: int64 4
0          M
1          M
2          M
3          M
4          M
       ...  
163    8-8.5
164    8-8.5
165    8-8.5
166    8-8.5
167    8-8.5
Name: Size, Length: 168, dtype: object 8


## Plot vertical heights

In [79]:
colors={
    "None": "black",
    "<i>Durex</i><br>Condom": "blue",
    "<i>Lifestyle</i><br>Condom": "blue",
    "<i>Skyn</i><br>Condom": "blue",
    "<i>Trojan</i><br>Condom": "blue",
    "Medium<br><i>Vinyl</i><br>Glove": "orange",
    "<i>Large</i><br>Nitrile<br>Glove": "orange",
    "Medium<br>Nitrile<br>Glove": "orange",
    "Medium<br>Nitrile<br>Glove,<br><i>2 fingers</i>": "orange",
    "<i>Small</i><br>Nitrile<br>Glove": "orange",
    "<i>Small</i><br>Nitrile<br>Glove,<br><i>Palm</i>": "orange",
    "<i>Medium</i><br>Nitrile<br>Glove,<br><i>Palm</i>": "orange",
}

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [108]:
!git status

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Refresh index: 100% (838/838), done.
On branch main
Your branch is based on 'origin/main', but the upstream is gone.
  (use "git branch --unset-upstream" to fixup)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   .gitattributes
	modified:   .idea/SpeculumAnalysis.iml
	deleted:    .idea/markdown.xml
	modified:   .idea/misc.xml
	new file:   data/01_raw/SpeculumTrialData_old.xlsx
	new file:   data/01_raw/SpeculumTrialData_v2 copy.xlsx
	renamed:    data/01_raw/SpeculumTrialData_v2_1.xlsx -> data/01_raw/SpeculumTrialData_v2.xlsx
	deleted:    data/01_raw/SpeculumTrialData_v2_2.5.xlsx
	deleted:    data/01_raw/SpeculumTrialData_v2_2.xlsx
	deleted:    data/01_raw/SpeculumTrialData_v2_3.xlsx
	modified:   data/02_intermediate/labels_df.csv
	modified:   data/03_processed/combined_df_long.csv
	modified:   data/03_processed/combined_df_long.pkl
	modified:   data/03_processed/combined_df_long.xlsx
	modified:   data/03_processed/combined_df_multiindex.pkl
	mod